<a href="https://colab.research.google.com/github/Pratham-Anand/streamlit-example/blob/master/kaggle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing utility modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
# importing machine learning models for prediction
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
# importing voting classifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier






In [ ]:
df=pd.read_csv("train.csv")
print(df.isna().sum(axis=0))

# getting target data from the dataframe
target=df.target
# getting train data from the dataframe
train =df.loc[ : ,df.columns!='target']

#test data
Y=pd.read_csv("test.csv")
Y_test=df.loc[ : ,  Y.columns!='id']

# Splitting between train data into training and validation dataset
x_train, x_test, y_train, y_test = train_test_split(
	train, target, train_size=0.99,test_size=0.001)


target    0
F1        0
F2        0
F3        0
F4        0
         ..
F1046     0
F1047     0
F1048     0
F1049     0
F1050     0
Length: 1051, dtype: int64


In [ ]:
model_1 = KNeighborsClassifier(3333)
model_2 = GaussianNB()
model_3 = RandomForestClassifier(10)
model_4 =  DecisionTreeClassifier(max_depth=10)
# Making the final model using voting classifier
final_model = VotingClassifier(
	estimators=[('KNN', model_1), ('NB',model_2), ('RF', model_3),('DT',model_4)],voting='soft')

# training all the model on the train dataset
final_model.fit(x_train, y_train)


VotingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=3333)),
                             ('NB', GaussianNB()),
                             ('RF', RandomForestClassifier(n_estimators=10)),
                             ('DT', DecisionTreeClassifier(max_depth=10))],
                 voting='soft')

In [ ]:
predicted_final = final_model.predict(Y_test)
# score = accuracy_score(y_test, y_pred)
print(predicted_final)


[1 3 3 ... 3 3 0]


In [ ]:
finalfile=pd.DataFrame({"id":Y.id,"target":predicted_final})
finalfile.to_csv('finaltarget.csv',index=False)

Parameter Tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train,y_train)                                

Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: ignored